In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression as LR

In [96]:
data=pd.read_csv(r"C:\Users\divyam.arora\Desktop\New Folder 3\machine-learning-specialization-master\course-3\datasets\amazon_baby_subset.csv",sep=',')

In [97]:
data.head()

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1


In [98]:
data['sentiment'].value_counts()

 1    26579
-1    26493
Name: sentiment, dtype: int64

In [99]:
import string
data['review'].fillna(value='',inplace=True)

In [100]:
data['review']=data['review'].astype(str)
def remove_punctuation(text):
    return text.translate(str.maketrans('','',string.punctuation)) 

data['review_clean']=data['review'].apply(remove_punctuation)

In [101]:
important_words=["baby", "one", "great", "love", "use", "would", "like", "easy", "little", "seat", "old", "well", "get", "also", "really", "son", "time", "bought", "product", "good", "daughter", "much", "loves", "stroller", "put", "months", "car", "still", "back", "used", "recommend", "first", "even", "perfect", "nice", "bag", "two", "using", "got", "fit", "around", "diaper", "enough", "month", "price", "go", "could", "soft", "since", "buy", "room", "works", "made", "child", "keep", "size", "small", "need", "year", "big", "make", "take", "easily", "think", "crib", "clean", "way", "quality", "thing", "better", "without", "set", "new", "every", "cute", "best", "bottles", "work", "purchased", "right", "lot", "side", "happy", "comfortable", "toy", "able", "kids", "bit", "night", "long", "fits", "see", "us", "another", "play", "day", "money", "monitor", "tried", "thought", "never", "item", "hard", "plastic", "however", "disappointed", "reviews", "something", "going", "pump", "bottle", "cup", "waste", "return", "amazon", "different", "top", "want", "problem", "know", "water", "try", "received", "sure", "times", "chair", "find", "hold", "gate", "open", "bottom", "away", "actually", "cheap", "worked", "getting", "ordered", "came", "milk", "bad", "part", "worth", "found", "cover", "many", "design", "looking", "weeks", "say", "wanted", "look", "place", "purchase", "looks", "second", "piece", "box", "pretty", "trying", "difficult", "together", "though", "give", "started", "anything", "last", "company", "come", "returned", "maybe", "took", "broke", "makes", "stay", "instead", "idea", "head", "said", "less", "went", "working", "high", "unit", "seems", "picture", "completely", "wish", "buying", "babies", "won", "tub", "almost", "either"]


In [102]:
for word in important_words:
    data[word] = data['review_clean'].apply(lambda s : s.split().count(word))

In [103]:
data.shape

(53072, 198)

In [104]:
features=data.columns.tolist()

In [105]:
features.remove('sentiment')

In [106]:
features.remove('name')
features.remove('review')
features.remove('rating')
features.remove('review_clean')

In [108]:
label=data['sentiment']
features=['constant'] + features
data['constant'] = 1

In [109]:
train_data, validation_data,train_label,validation_label=train_test_split(data[features].as_matrix(),label.as_matrix(),test_size=0.2,random_state=3)

In [32]:
len(data[data['perfect']>0])

2955

In [33]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = features_frame.as_matrix()
    label_sarray = dataframe[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

In [110]:
#link function for logistic regression
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score=np.dot(feature_matrix,coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1/(1+np.exp(-score))
    
    # return predictions
    return predictions

In [111]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])

correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),          1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_predictions = np.array( [ 1./(1+np.exp(-correct_scores[0])), 1./(1+np.exp(-correct_scores[1])) ] )

print('The following outputs must match ')
print('------------------------------------------------')
print('correct_predictions           =', correct_predictions)
print('output of predict_probability =', predict_probability(dummy_feature_matrix, dummy_coefficients))

The following outputs must match 
------------------------------------------------
correct_predictions           = [ 0.98201379  0.26894142]
output of predict_probability = [ 0.98201379  0.26894142]


In [112]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    value=np.dot(errors,feature)
    derivative = np.sum(value)
    
    # Return the derivative
    return derivative

In [113]:
#the above is noraml likelyhood function
#but better is log likelyhood for numerical calculations
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
        indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]
    
    lp = np.sum((indicator-1)*scores - logexp)
    return lp

In [114]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])
dummy_sentiment = np.array([-1, 1])

correct_indicators  = np.array( [ -1==+1,                                       1==+1 ] )
correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),                     1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_first_term  = np.array( [ (correct_indicators[0]-1)*correct_scores[0],  (correct_indicators[1]-1)*correct_scores[1] ] )
correct_second_term = np.array( [ np.log(1. + np.exp(-correct_scores[0])),      np.log(1. + np.exp(-correct_scores[1])) ] )

correct_ll          =      sum( [ correct_first_term[0]-correct_second_term[0], correct_first_term[1]-correct_second_term[1] ] ) 

print('The following outputs must match ')
print('------------------------------------------------')
print('correct_log_likelihood           =', correct_ll)
print('output of compute_log_likelihood =', compute_log_likelihood(dummy_feature_matrix, dummy_sentiment, dummy_coefficients))

The following outputs must match 
------------------------------------------------
correct_log_likelihood           = -5.33141161544
output of compute_log_likelihood = -5.33141161544


In [115]:
#logistic regression
from math import sqrt

def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):

        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in range(len(coefficients)): # loop over each coefficient
            
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])
            
            # add the step size times the derivative to the current coefficient
            ## YOUR CODE HERE
            coefficients[j]=coefficients[j]+(derivative*step_size)
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [116]:
coefficients = logistic_regression(train_data, train_label, initial_coefficients=np.zeros(194),
                                   step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -29425.21708124
iteration   1: log likelihood of observed labels = -29421.48773918
iteration   2: log likelihood of observed labels = -29417.76180887
iteration   3: log likelihood of observed labels = -29414.03928044
iteration   4: log likelihood of observed labels = -29410.32014403
iteration   5: log likelihood of observed labels = -29406.60438991
iteration   6: log likelihood of observed labels = -29402.89200836
iteration   7: log likelihood of observed labels = -29399.18298977
iteration   8: log likelihood of observed labels = -29395.47732457
iteration   9: log likelihood of observed labels = -29391.77500328
iteration  10: log likelihood of observed labels = -29388.07601645
iteration  11: log likelihood of observed labels = -29384.38035472
iteration  12: log likelihood of observed labels = -29380.68800879
iteration  13: log likelihood of observed labels = -29376.99896943
iteration  14: log likelihood of observed labels = -29373.3132

In [117]:
train_data.shape

(42457, 194)

In [118]:
#now calculating class predictions from the scores
scores = np.dot(train_data, coefficients)

In [119]:
output=[+1 if x > 0 else -1 for x in scores]

In [120]:
count=0
count2=0
for v in output:
    if v>0:
        count=count+1
    else:
        count2=count2+1
    

In [121]:
print('positive ',count,' negative ',count2)

positive  19084  negative  23373


In [122]:
num_mistakes = np.sum(output!=train_label)
accuracy = (len(train_data) - num_mistakes)/len(train_label)
print("-----------------------------------------------------")
print('# Reviews   correctly classified =', len(train_data) - num_mistakes)
print('# Reviews incorrectly classified =', num_mistakes)
print('# Reviews total                  =', len(train_data))
print("-----------------------------------------------------")
print('Accuracy = %.2f' % accuracy)

-----------------------------------------------------
# Reviews   correctly classified = 31893
# Reviews incorrectly classified = 10564
# Reviews total                  = 42457
-----------------------------------------------------
Accuracy = 0.75


In [123]:
num_mistakes

10564

In [124]:
len(train_label)

42457

In [125]:
len(coefficients)

194

In [126]:
#coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

In [127]:
word_coefficient_tuples

[('use', 0.053424019750150077),
 ('love', 0.052025375371490108),
 ('little', 0.051570982457664866),
 ('stroller', 0.036217720043593207),
 ('seat', 0.036059310657732067),
 ('get', 0.0243718290206698),
 ('nice', 0.024253874608899322),
 ('well', 0.01612870682325495),
 ('bag', 0.0151341814052819),
 ('much', 0.014647285077178449),
 ('since', 0.013633254235782277),
 ('comfortable', 0.013630680843868334),
 ('see', 0.013221529156732335),
 ('first', 0.012882453647051999),
 ('bottles', 0.01194637558580978),
 ('one', 0.011169343796824105),
 ('really', 0.011095990351755399),
 ('toy', 0.01095022606499812),
 ('time', 0.010538278555450644),
 ('still', 0.01049079869372708),
 ('night', 0.0088694692513213002),
 ('made', 0.0088162493690133095),
 ('way', 0.008690670942880882),
 ('small', 0.0083515290463725116),
 ('put', 0.0080535244682797356),
 ('go', 0.0078140780410235339),
 ('works', 0.0077694657908520945),
 ('day', 0.0076495642113183649),
 ('bit', 0.0073099530008104846),
 ('think', 0.006487349332596277

In [128]:
word_coefficient_tuples_2 = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=False)

In [90]:
word_coefficient_tuples_2

[('would', -0.046074379571151368),
 ('product', -0.034451264008009574),
 ('money', -0.031258669955707939),
 ('work', -0.027872463691569756),
 ('even', -0.024431167282367428),
 ('get', -0.024380859380652056),
 ('disappointed', -0.023428160808205905),
 ('back', -0.023102381747833904),
 ('return', -0.021169458551753655),
 ('monitor', -0.019765497458497167),
 ('waste', -0.019180742684257065),
 ('thought', -0.017557775957254957),
 ('could', -0.01686042964068676),
 ('tried', -0.016078369243923959),
 ('time', -0.014937110634026323),
 ('buy', -0.014827329720047815),
 ('way', -0.014707642942655074),
 ('item', -0.01350141349456362),
 ('returned', -0.01314295898029174),
 ('broke', -0.012670014102693526),
 ('better', -0.012182204843180175),
 ('reviews', -0.012128670841533466),
 ('thing', -0.011915203124079),
 ('plastic', -0.011739400961077449),
 ('cheap', -0.011584295114563603),
 ('hard', -0.011456234083391246),
 ('something', -0.01117032636945345),
 ('first', -0.011166968813312738),
 ('bad', -0.0